In [ ]:
require 'httpclient'
require 'rest-client'
require 'json'

config = {
  scheme: 'http',
  hostname: 'localhost',
  port: 8080
  }

admin = {
  username: 'admin@everest.engineering',
  password: 'ac0n3x72'
  }

org1_user1 = {
  username: 'org1user1@acme.org',
  password: 'justauser'
  }


# System

## Version info


In [ ]:
response = RestClient.get "#{config[:scheme]}://#{config[:hostname]}:#{config[:port]}/api/version"
response.body

# Admin

## Admin login


In [ ]:
response = RestClient.post "#{config[:scheme]}://#{config[:hostname]}:#{config[:port]}/oauth/token?grant_type=password&client_id=web-app-ui&username=#{admin[:username]}&password=#{admin[:password]}", {}
admin_token = JSON.parse(response.body)["access_token"]

## Create organisation


In [ ]:
response = RestClient.post("#{config[:scheme]}://#{config[:hostname]}:#{config[:port]}/api/organizations",
{
  organizationName: "ACME Corporation"
}.to_json,
{
  content_type: :json,
  Authorization: "Bearer #{admin_token}"
})
orgId = JSON.parse(response.body)

## List organisations

In [ ]:
response = RestClient.get("#{config[:scheme]}://#{config[:hostname]}:#{config[:port]}/api/organizations",
{
  content_type: :json,
  Authorization: "Bearer #{admin_token}"
})
orgList = JSON.parse(response.body)

## Create organisation user

In [ ]:
response = RestClient.post("#{config[:scheme]}://#{config[:hostname]}:#{config[:port]}/api/organizations/#{orgId}/users",
{
  username: "#{org1_user1[:username]}",
  displayName: "Loyal employee",
  password: "#{org1_user1[:password]}"
}.to_json,
{
  content_type: :json,
  Authorization: "Bearer #{admin_token}"
})
org1_user1_jwt = JSON.parse(response.body)

## List organisation users

In [ ]:
response = RestClient.get("#{config[:scheme]}://#{config[:hostname]}:#{config[:port]}/api/organizations/#{orgId}/users",
{
  content_type: :json,
  Authorization: "Bearer #{admin_token}"
})
orgAdmins = JSON.parse(response.body)

# User


## Login as organisation user

In [ ]:
response = RestClient.post "#{config[:scheme]}://#{config[:hostname]}:#{config[:port]}/oauth/token?grant_type=password&client_id=web-app-ui&username=#{org1_user1[:username]}&password=#{org1_user1[:password]}", {}
org1_user1_jwt = JSON.parse(response.body)["access_token"]

## Upload profile photo

In [ ]:
response = RestClient.post("#{config[:scheme]}://#{config[:hostname]}:#{config[:port]}/api/user/profile-photo",
{
  file: File.new("./images/profile.jpg", 'rb'),
},
{
  
  Authorization: "Bearer #{org1_user1_jwt}"
})

## Download (stream) profile photo thumbnail

In [ ]:
response = RestClient.get("#{config[:scheme]}://#{config[:hostname]}:#{config[:port]}/api/user/profile-photo/thumbnail?width=127&height=128",
{
  content_type: :json,
  Authorization: "Bearer #{org1_user1_jwt}"
})
File.open("/tmp/downloaded-profile-thumbnail.jpg", "w") {|f| f.write(response.to_s) }
File.open("/tmp/downloaded-profile-thumbnail.jpg")